In [1]:
!pip install yfinance --upgrade --no-cache-dir

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=cfb14d447b0b20439afd6813b71d03671e76182e1c780e19cbeb7f438659f7e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-pikbw7qz/wheels/6f/ad/f4/4a269deab015672fd1ab353d6b2c3fcf64f413980737c13541
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8366 sha256=0c71672ce14d77eeb02689b0e12e47ebac55a0b57d4b98c2ac36cd538ce9204a
  Stored in directory: /tmp/pip-ephem-wheel-cache-pikbw7qz/wheels/5e/8a/c4/59c699498647c7c94b14c87a904ca7540646107b3d94b7c320
Successfully built yfinance multitasking
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import bs4 as bs
import requests
import yfinance as yf
import datetime
import pandas as pd
import matplotlib.pyplot as plt

## Step1: Get stock tickers in S&P 500

First we scrape the wiki page http://en.wikipedia.org/wiki/List_of_S%26P_500_companies to get all stock tickers in S&P 500 as the stock universe. 

In [3]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

tickers = [s.replace('\n', '') for s in tickers]
print(tickers)

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'ADS', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BK', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIIB', 'BLK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF.B', 'CHRW', 'COG', 'CDNS', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'CXO', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'COTY', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA

## Step2: Acquire adjusted close prices

We want to download the adjusted close prices for all stocks in S&P 500 starting from January 1 2006 to June 9 2020. 

In [4]:
start = datetime.datetime(2006,1,1)
end = datetime.datetime(2020,6,9)

We use yfinance (https://pypi.org/project/yfinance/) to download the price data given all the tickers in S&P 500.

In [5]:
data = yf.download(tickers, start=start, end=end)

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


In [6]:
data.head()

Adj Close                                                         \
                    A        AAL        AAP      AAPL ABBV        ABC   ABMD   
Date                                                                           
2006-01-03  20.609097  35.305672  41.206989  9.244411  NaN  16.633608   9.35   
2006-01-04  20.664463  36.955460  41.500534  9.271619  NaN  16.523405   9.62   
2006-01-05  21.205837  37.436264  41.699368  9.198654  NaN  16.318720   9.55   
2006-01-06  21.316574  36.766911  41.652027  9.436100  NaN  16.137671   9.75   
2006-01-09  21.255054  36.399246  42.191738  9.405183  NaN  16.137671  10.15   

                                             ...      Volume              \
                  ABT        ACN       ADBE  ...        XLNX         XOM   
Date                                         ...                           
2006-01-03  10.491420  22.193466  38.520000  ...   9196200.0  23871000.0   
2006-01-04  10.507332  22.314663  38.419998  ...  10421200.0  19086500.0   
2006-01-05  10.642622  22.481304  38.070000  ...  20062800.0  17158400.0   
2006-01-06  10.846883  23.594763  39.000000  ...   7574200.0  17368600.0   
2006-01-09  11.250092  23.526592  38.380001  ...   3685400.0  17339100.0   

                                                                      \
                 XRAY        XRX XYL        YUM        ZBH      ZBRA   
Date                                                                   
2006-01-03  1054800.0  1340600.0 NaN  4135700.0  1580700.0  618800.0   
2006-01-04   809200.0  1191400.0 NaN  7202800.0  1080200.0  444300.0   
2006-01-05  1080800.0   867300.0 NaN  7962900.0  1199400.0  313000.0   
2006-01-06   873800.0  1153700.0 NaN  7015600.0  1348800.0  617100.0   
2006-01-09   989400.0  1335200.0 NaN  4077000.0  1321400.0  310700.0   

                          
                ZION ZTS  
Date                      
2006-01-03  663700.0 NaN  
2006-01-04  523100.0 NaN  
2006-01-05  535800.0 NaN  
2006-01-06  544600.0 NaN  
2006-01-09  639700.0 NaN  

[5 rows x 3030 columns]

Now, we need to extract 'Adj Close' column and do a bit data cleaning. 

In [7]:
# Columns are multi-indexed. Here we just extract Adj Close
adj_close = data.loc[:, 'Adj Close']
# Propagate last valid observation forward to next valid
adj_close = adj_close.fillna(method='ffill')
# Drop stocks have NaN values
adj_close = adj_close.dropna(axis = 1)

In [8]:
adj_close.head()

,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADBE,ADI,...,WYNN,XEL,XLNX,XOM,XRAY,XRX,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2006-01-03,20.609097,35.305672,41.206989,9.244411,16.633608,9.35,10.491420,22.193466,38.520000,25.182011,...,29.771486,10.504530,18.621290,37.851017,24.792299,32.808895,11.401830,63.671101,42.830002,62.799614
2006-01-04,20.664463,36.955460,41.500534,9.271619,16.523405,9.62,10.507332,22.314663,38.419998,25.415308,...,29.716314,10.555444,19.459511,37.915764,25.021738,32.764881,11.499380,64.275429,42.410000,63.243000
2006-01-05,21.205837,37.436264,41.699368,9.198654,16.318720,9.55,10.642622,22.481304,38.070000,26.224972,...,29.539719,10.549784,20.630112,37.728016,24.737236,32.412811,11.989601,63.596718,42.529999,63.817776
2006-01-06,21.316574,36.766911,41.652027,9.436100,16.137671,9.75,10.846883,23.594763,39.000000,26.327896,...,29.821154,10.589380,21.041996,38.472500,25.122686,32.214756,11.955452,63.271294,44.119999,64.482903
2006-01-09,21.255054,36.399246,42.191738,9.405183,16.137671,10.15,11.250092,23.526592,38.380001,26.698418,...,31.228340,10.561097,21.020315,38.453083,25.370464,32.654846,12.048132,65.512039,44.790001,63.949154


In [9]:
adj_close.tail()

,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADBE,ADI,...,WYNN,XEL,XLNX,XOM,XRAY,XRX,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2020-06-02,90.290001,11.220000,139.241592,323.339996,98.660004,230.660004,92.620003,205.779999,390.440002,116.800003,...,84.099998,65.633461,93.099998,47.310001,47.369999,17.209999,92.209999,128.479996,264.089996,33.450001
2020-06-03,90.489998,11.850000,142.066559,325.119995,99.489998,238.039993,91.400002,207.279999,389.920013,121.620003,...,93.500000,66.378502,95.389999,49.240002,47.840000,17.950001,94.559998,130.369995,269.019989,35.490002
2020-06-04,91.139999,16.719999,141.946762,322.320007,99.449997,236.020004,88.250000,203.100006,385.799988,122.379997,...,95.379997,64.600327,95.680000,49.099998,48.040001,17.680000,94.989998,129.509995,261.170013,37.230000
2020-06-05,90.379997,18.590000,145.839828,331.500000,98.970001,249.809998,89.860001,208.490005,392.899994,124.589996,...,101.610001,64.828812,96.199997,53.080002,50.180000,18.459999,96.519997,137.399994,277.549988,38.610001
2020-06-08,90.290001,20.309999,146.089371,333.459991,102.300003,253.300003,92.559998,211.279999,397.779999,123.709999,...,108.800003,66.507645,96.540001,54.740002,51.169998,19.870001,95.989998,136.500000,277.059998,40.570000


In [10]:
adj_close.columns.values

array(['A', 'AAL', 'AAP', 'AAPL', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE',
       'ADI', 'ADM', 'ADP', 'ADS', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL',
       'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL',
       'ALXN', 'AMAT', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANSS',
       'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'ARE', 'ATO', 'ATVI',
       'AVB', 'AVY', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX',
       'BEN', 'BIIB', 'BK', 'BKNG', 'BKR', 'BLK', 'BLL', 'BMY', 'BSX',
       'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CAT', 'CB', 'CBRE', 'CCI', 'CCL',
       'CDNS', 'CE', 'CERN', 'CF', 'CHD', 'CHRW', 'CI', 'CINF', 'CL',
       'CLX', 'CMA', 'CMCSA', 'CME', 'CMI', 'CMS', 'CNC', 'CNP', 'COF',
       'COG', 'COO', 'COP', 'COST', 'CPB', 'CPRT', 'CRM', 'CSCO', 'CSX',
       'CTAS', 'CTL', 'CTSH', 'CTXS', 'CVS', 'CVX', 'D', 'DD', 'DE',
       'DGX', 'DHI', 'DHR', 'DIS', 'DISCA', 'DISH', 'DLR', 'DLTR', 'DOV',
       'DPZ', 'DRE', 'DRI', 'DTE', 'DUK', 'DVA', 'DVN', 'DXC', 'DX

In [11]:
print(f'Number of missing values: {adj_close.isnull().sum().sum()}')

Number of missing values: 0


In [12]:
print(adj_close.describe())

                 A          AAL          AAP         AAPL          ABC  \
count  3638.000000  3638.000000  3638.000000  3638.000000  3638.000000   
mean     37.095672    25.725284    92.262614    84.788635    51.454688   
std      19.458048    16.203190    49.862242    72.144847    29.572422   
min       8.122367     1.659225    23.294809     6.266412    11.512946   
25%      22.460566     9.033596    40.496697    23.016733    20.564123   
50%      30.384507    28.426829    80.480515    67.367977    45.097704   
75%      45.125700    39.911220   143.888115   116.669054    81.516388   
max      91.139999    59.345577   198.015549   333.459991   105.652664   

              ABMD          ABT          ACN         ADBE          ADI  ...  \
count  3638.000000  3638.000000  3638.000000  3638.000000  3638.000000  ...   
mean     78.523560    34.272205    76.867654    92.661572    47.265778  ...   
std     102.582496    21.052342    52.250997    89.347635    28.603947  ...   
min       4.90000

In [13]:
adj_close.to_csv('data/adj_close.csv')